In [1]:
import snowflake
from snowflake.snowpark import Session
import os
from dotenv import load_dotenv

In [4]:
# Load environment variables from .env file
load_dotenv()
account = os.getenv('ACCOUNT')
password =os.getenv('PASSWORD')
user = os.getenv('USER')
warehouse = os.getenv('WAREHOUSE')
database = os.getenv('DATABASE')
schema = os.getenv('SCHEMA')

In [5]:
connections = snowflake.connector.connect(user = user,
                                  password = password,
                                  account = account,
                                  database = database,
                                  schema = schema)

In [6]:
cursor = connections.cursor()

In [8]:
cursor.execute('Show schemas;')

In [9]:
for i in cursor:
    print(i)

(datetime.datetime(2024, 9, 8, 23, 11, 45, 101000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'INFORMATION_SCHEMA', 'N', 'N', 'SNOWPARK_DEMO_DB', '', 'Views describing the contents of schemas in this database', '', '1', '', None)
(datetime.datetime(2024, 9, 5, 0, 2, 16, 831000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'PUBLIC', 'N', 'N', 'SNOWPARK_DEMO_DB', 'ACCOUNTADMIN', '', '', '1', 'ROLE', None)
(datetime.datetime(2024, 9, 5, 0, 2, 17, 657000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'SNOWPARK_DEMO_SCHEMA', 'N', 'Y', 'SNOWPARK_DEMO_DB', 'ACCOUNTADMIN', '', '', '1', 'ROLE', None)


In [10]:
cursor.execute('Use warehouse SQL_QUERY_WH;')


In [12]:
cursor.execute('select current_warehouse();')

In [13]:
for i in cursor:
    print(i)

('SQL_QUERY_WH',)


In [14]:
cursor.execute('Use database SAMPLE_DATABASE;')

In [15]:
cursor.execute('Select current_database();')
for db in cursor:
    print(db)

('SAMPLE_DATABASE',)


In [17]:
cursor.execute('use schema MY_SCHEMA;')
cursor.execute('Select current_schema();')
for db in cursor:
    print(db)

('MY_SCHEMA',)


# Create table

In [23]:
cursor.execute('CREATE TABLE LUCKY(SALARY INT, WEIGHT DECIMAL(5, 3), INSURANCE BOOLEAN);')


In [24]:
cursor.execute("Show tables;")

In [25]:
for table in cursor:
    print(table)

(datetime.datetime(2024, 9, 8, 23, 49, 55, 146000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'LUCKY', 'SAMPLE_DATABASE', 'MY_SCHEMA', 'TABLE', '', '', 0, 0, 'ACCOUNTADMIN', '1', 'OFF', 'OFF', 'OFF', None, None, 'N', 'N', 'ROLE', 'N', None, 'N', 'N', 'N', 'N')


# Insert 

In [26]:
cursor.execute('INSERT INTO LUCKY(SALARY, WEIGHT, INSURANCE) VALUES(1234, 12.23422, TRUE);')

In [27]:
cursor.execute('INSERT INTO LUCKY(SALARY, WEIGHT, INSURANCE) VALUES(1235, 14.23422, 1);')

In [28]:
cursor.execute('INSERT INTO LUCKY(SALARY, WEIGHT, INSURANCE) VALUES(12344, 12.23422, 0);')

In [34]:
cursor.execute('INSERT INTO LUCKY(SALARY, WEIGHT, INSURANCE) VALUES(1234, 12.23422,  FALSE);')

In [35]:
cursor.execute('INSERT INTO LUCKY VALUES(salary= 3453, "weight" =12.23422, "insurance" =TRUE);')

ProgrammingError: 000904 (42000): SQL compilation error: error line 1 at position 25
invalid identifier 'SALARY'

In [37]:
cursor.execute('SELECT * FROM LUCKY;')
data = cursor.fetchall()
data

[(1234, Decimal('12.234'), True),
 (1235, Decimal('14.234'), True),
 (12344, Decimal('12.234'), False),
 (1234, Decimal('12.234'), False)]

# Alter table

In [41]:
query = 'ALTER TABLE LUCKY ADD COVID VARCHAR(5);'
cursor.execute(query)

ProgrammingError: 001430 (42601): SQL compilation error:
column 'COVID' already exists

In [47]:
query = 'UPDATE LUCKY SET COVID = yesss where SALARY = 1234;'
cursor.execute(query)

ProgrammingError: 000904 (42000): SQL compilation error: error line 1 at position 25
invalid identifier 'YESSS'

# date and time

In [50]:
cursor.execute('CREATE TABLE DATE (DATEIN DATE);')

In [51]:
cursor.execute('INSERT INTO DATE (DATEIN) values ( CURDATE());')

ProgrammingError: 002140 (42601): SQL compilation error:
Unknown function CURDATE

In [52]:
cursor.execute('INSERT INTO DATE (DATEIN) values ( CURRENT_DATE());')

In [53]:
cursor.execute('INSERT INTO DATE (DATEIN) values ( CURRENT_TIMESTAMP());')

In [54]:
cursor.execute('INSERT INTO DATE (DATEIN) values ( CURRENT_TIME());')

ProgrammingError: 002023 (22000): SQL compilation error:
Expression type does not match column data type, expecting DATE but got TIME(9) for column DATEIN

In [55]:
cursor.execute('INSERT INTO DATE (DATEIN) values ( NOW());')

ProgrammingError: 002140 (42601): SQL compilation error:
Unknown function NOW

In [57]:
cursor.execute('insert into date (datein) values (24:09:09);')

ProgrammingError: 001003 (42000): SQL compilation error:
syntax error line 1 at position 37 unexpected '09'.
syntax error line 1 at position 40 unexpected '09'.

In [ ]:

# https://docs.snowflake.com/en/sql-reference/date-time-examples
SELECT CURRENT_DATE();

SELECT CURRENT_TIMESTAMP();

In [77]:
cursor.execute("INSERT INTO date (datein) VALUES (TO_DATE('2024,01,08', 'YYYY,MM,DD'))")

In [76]:
cursor.execute("INSERT INTO date (datein) VALUES (TO_DATE('2024:01:08', 'YYYY:MM:DD'))")

In [60]:
cursor.execute('SELECT * FROM date;')
data = cursor.fetchall()
data

[(datetime.date(2024, 9, 9),), (datetime.date(2024, 9, 9),)]

In [66]:
query = 'ALTER TABLE DATE ADD YEAR INT;'
cursor.execute(query)

In [67]:
query = 'update date set year = year(datein);'
cursor.execute(query)

In [68]:
cursor.execute('SELECT * FROM date;')
data = cursor.fetchall()
data

[(datetime.date(2024, 9, 9), 2024), (datetime.date(2024, 9, 9), 2024)]

In [69]:
query = 'ALTER TABLE DATE ADD MONTH INT, DATE INT;'
cursor.execute(query)

In [70]:
query = 'update date set MONTH = month(datein), date = day(datein);'
cursor.execute(query)

In [71]:
cursor.execute('SELECT * FROM date;')
data = cursor.fetchall()
data

[(datetime.date(2024, 9, 9), 2024, 9, 9),
 (datetime.date(2024, 9, 9), 2024, 9, 9)]

In [72]:
cursor.execute("INSERT INTO date (datein) VALUES ('2014-09-16' )")

In [73]:
cursor.execute('SELECT * FROM date;')
data = cursor.fetchall()
data

[(datetime.date(2024, 9, 9), 2024, 9, 9),
 (datetime.date(2024, 9, 9), 2024, 9, 9),
 (datetime.date(2014, 9, 16), None, None, None)]

In [84]:
cursor.execute("INSERT INTO date (datein) VALUES (TO_DATE('2025:01:08', 'YYYY:MM:DD')), (TO_DATE('2022:01:08', 'YYYY:MM:DD'));")

In [85]:
cursor.execute('SELECT * FROM date;')
data = cursor.fetchall()
data

[(datetime.date(2024, 9, 9), 2024, 9, 9),
 (datetime.date(2024, 9, 9), 2024, 9, 9),
 (datetime.date(2014, 9, 16), None, None, None),
 (datetime.date(2024, 1, 8), None, None, None),
 (datetime.date(2024, 1, 8), None, None, None),
 (datetime.date(2025, 1, 8), None, None, None),
 (datetime.date(2022, 1, 8), None, None, None)]

In [74]:
cursor.execute('INSERT INTO DATE (DATEIN) values ( DATE());')

ProgrammingError: 001044 (42P13): SQL compilation error: error line 1 at position 35
Invalid argument types for function 'DATE': ()

In [64]:
cursor.execute("CREATE TABLE datetimedetails (datein DATETIME)")

In [65]:
cursor.execute("INSERT INTO datetimedetails (datein) VALUES ( CURDATE())")

ProgrammingError: 002140 (42601): SQL compilation error:
Unknown function CURDATE

# DIABETES DATAFRAME

In [78]:
import pandas as pd

In [79]:
df = pd.read_csv('diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [80]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [83]:
df.to_sql('diabetes', con = cursor)

C:\Users\user\AppData\Local\Temp\ipykernel_7596\2983270208.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('diabetes', con = cursor)


AttributeError: 'SnowflakeCursor' object has no attribute 'cursor'

In [89]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [97]:
print(df.columns)

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [93]:
datas = str([tuple(data) for data in df.values.tolist()])[1:-1]

In [99]:
query = '''CREATE TABLE DIABETES (Pregnancies INT, 
            Glucose INT, 
            BloodPressure INT,
            SkinThickness INT,
            Insulin INT,
            BMI DECIMAL,
            DiabetesPedigreeFunction  DECIMAL,
            Age INT,
            Outcome INT);'''

cursor.execute(query)

In [100]:
query = f'INSERT INTO DIABETES VALUES {datas}'
query

'INSERT INTO DIABETES VALUES (6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0, 1.0), (1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0, 0.0), (8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0, 1.0), (1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0, 0.0), (0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0, 1.0), (5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0, 0.0), (3.0, 78.0, 50.0, 32.0, 88.0, 31.0, 0.248, 26.0, 1.0), (10.0, 115.0, 0.0, 0.0, 0.0, 35.3, 0.134, 29.0, 0.0), (2.0, 197.0, 70.0, 45.0, 543.0, 30.5, 0.158, 53.0, 1.0), (8.0, 125.0, 96.0, 0.0, 0.0, 0.0, 0.232, 54.0, 1.0), (4.0, 110.0, 92.0, 0.0, 0.0, 37.6, 0.191, 30.0, 0.0), (10.0, 168.0, 74.0, 0.0, 0.0, 38.0, 0.537, 34.0, 1.0), (10.0, 139.0, 80.0, 0.0, 0.0, 27.1, 1.441, 57.0, 0.0), (1.0, 189.0, 60.0, 23.0, 846.0, 30.1, 0.398, 59.0, 1.0), (5.0, 166.0, 72.0, 19.0, 175.0, 25.8, 0.587, 51.0, 1.0), (7.0, 100.0, 0.0, 0.0, 0.0, 30.0, 0.484, 32.0, 1.0), (0.0, 118.0, 84.0, 47.0, 230.0, 45.8, 0.551, 31.0, 1.0), (7.0, 107.0, 74.0, 0.0, 0.0, 29.6, 

In [101]:
cursor.execute(query)

In [102]:
cursor.execute('select * from diabetes;')
data = cursor.fetchall()

In [103]:
len(data)

768

In [104]:
pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,34,1,50,1
1,1,85,66,29,0,27,0,31,0
2,8,183,64,0,0,23,1,32,1
3,1,89,66,23,94,28,0,21,0
4,0,137,40,35,168,43,2,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,33,0,63,0
764,2,122,70,27,0,37,0,27,0
765,5,121,72,23,112,26,0,30,0
766,1,126,60,0,0,30,0,47,1


In [105]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


# SELECT QUERY

In [106]:
cursor.execute('SELECT * FROM diabetes limit 20;')
out = cursor.fetchall()


In [108]:
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,PREGNANCIES,GLUCOSE,BLOODPRESSURE,SKINTHICKNESS,INSULIN,BMI,DIABETESPEDIGREEFUNCTION,AGE,OUTCOME
0,6,148,72,35,0,34,1,50,1
1,1,85,66,29,0,27,0,31,0
2,8,183,64,0,0,23,1,32,1
3,1,89,66,23,94,28,0,21,0
4,0,137,40,35,168,43,2,33,1
5,5,116,74,0,0,26,0,30,0
6,3,78,50,32,88,31,0,26,1
7,10,115,0,0,0,35,0,29,0
8,2,197,70,45,543,31,0,53,1
9,8,125,96,0,0,0,0,54,1


In [110]:
cursor.execute('Select Age, BMI from diabetes limit 5')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,BMI
0,50,34
1,31,27
2,32,23
3,21,28
4,33,43


In [111]:
cursor.execute('select age, bmi, outcome from diabetes where outcome =1 limit 5;')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,BMI,OUTCOME
0,50,34,1
1,32,23,1
2,33,43,1
3,26,31,1
4,53,31,1


In [112]:
cursor.execute('select age, bmi, outcome from diabetes where outcome =1 and age>30 limit 5')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,BMI,OUTCOME
0,50,34,1
1,32,23,1
2,33,43,1
3,53,31,1
4,54,0,1


In [114]:
cursor.execute('select age, outcome from diabetes where outcome =1 and age >=34 and age <=35 limit 5')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,OUTCOME
0,34,1
1,35,1
2,35,1
3,35,1
4,35,1


In [115]:
cursor.execute('select age from diabetes where age between 35 and 36;')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE
0,35
1,36
2,36
3,36
4,36
5,36
6,36
7,36
8,36
9,35


In [116]:
cursor.execute('select count(pregnancies) from diabetes;')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,COUNT(PREGNANCIES)
0,768


In [117]:
cursor.execute('select * from diabetes where pregnancies = 1;')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,PREGNANCIES,GLUCOSE,BLOODPRESSURE,SKINTHICKNESS,INSULIN,BMI,DIABETESPEDIGREEFUNCTION,AGE,OUTCOME
0,1,85,66,29,0,27,0,31,0
1,1,89,66,23,94,28,0,21,0
2,1,189,60,23,846,30,0,59,1
3,1,103,30,38,83,43,0,33,0
4,1,115,70,30,96,35,1,32,1
...,...,...,...,...,...,...,...,...,...
130,1,121,78,39,74,39,0,28,0
131,1,128,88,39,110,37,1,37,1
132,1,106,76,0,0,38,0,26,0
133,1,126,60,0,0,30,0,47,1


In [118]:
cursor.execute('select count(*) as whole_count from diabetes where pregnancies = 2;')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,WHOLE_COUNT
0,103


In [119]:
cursor.execute('select count(*) as whole_count from diabetes where pregnancies in (1, 2);')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,WHOLE_COUNT
0,238


In [120]:
cursor.execute('select count(*) as whole_count from diabetes where pregnancies not in (1, 2, 3,4, 5);')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,WHOLE_COUNT
0,330


In [121]:
cursor.execute('select avg(bloodpressure) as Avg_BP, count(*) from diabetes where outcome = 1 and pregnancies >10;')
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AVG_BP,COUNT(*)
0,71.300000,20


In [123]:
query = """ select   count(pregnancies), sum(outcome) from diabetes
        where outcome = 1
        group by pregnancies, outcome"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,COUNT(PREGNANCIES),SUM(OUTCOME)
0,16,16
1,22,22
2,38,38
3,27,27
4,19,19
5,10,10
6,29,29
7,21,21
8,25,25
9,18,18


In [124]:

query = """ select  pregnancies, count(pregnancies), sum(outcome) from diabetes
        
        group by pregnancies, outcome"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])


,PREGNANCIES,COUNT(PREGNANCIES),SUM(OUTCOME)
0,6,16,16
1,1,106,0
2,8,22,22
3,0,38,38
4,5,36,0
5,3,27,27
6,10,14,0
7,2,19,19
8,4,45,0
9,10,10,10


In [125]:
query = """select min(pregnancies) as minpre, max(bloodpressure) from diabetes """
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,MINPRE,MAX(BLOODPRESSURE)
0,0,122


In [ ]:
curs.execute('select least(4,5,2,6,7)')
curs.execute('select power(4,5)')
curs.execute('select greatest(4,5)')
curs.execute('select floor(20.5)')
curs.execute('select ceiling(20.61)')
curs.execute('select sum(4+7)')
data=curs.fetchall()
pd.DataFrame(data, columns=[i[0] for i in curs.description])

In [126]:
query = """ select count(pregnancies) from diabetes where outcome =0 and bloodpressure between 40 and 50"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,COUNT(PREGNANCIES)
0,18


In [127]:
query = """ select count(pregnancies) from diabetes where outcome =0 and bloodpressure not between 40 and 50"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,COUNT(PREGNANCIES)
0,482


In [ ]:
query = """ """
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

In [128]:
query = """ select age, bloodpressure, insulin, outcome from diabetes where bloodpressure >100 and age in 
            (select age from diabetes where outcome = 1 and insulin between 80 and 120)"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,BLOODPRESSURE,INSULIN,OUTCOME
0,37,108,0,1
1,27,122,0,0
2,26,110,130,1
3,41,104,0,1
4,37,110,0,0
5,43,106,231,1
6,47,106,49,0


# partition

In [129]:
query = """select age, pregnancies, count(*) over (partition by pregnancies order by age desc) as countw
from diabetes """
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,PREGNANCIES,COUNTW
0,50,6,11
1,31,1,28
2,32,8,36
3,21,1,135
4,33,0,17
...,...,...,...
763,63,10,1
764,27,2,33
765,30,5,42
766,47,1,5


In [131]:
query = """select age, pregnancies, count(*) over (partition by pregnancies order by age desc) as countw
from diabetes where pregnancies <5"""
cursor.execute(query)
out = cursor.fetchall()
dff =pd.DataFrame(out, columns= [i[0] for i in cursor.description])
dff

,AGE,PREGNANCIES,COUNTW
0,31,1,28
1,21,1,135
2,33,0,17
3,26,3,43
4,53,2,6
...,...,...,...
487,26,1,56
488,22,2,89
489,27,2,33
490,47,1,5


In [133]:
dff[dff['AGE']==31]

,AGE,PREGNANCIES,COUNTW
0,31,1,28
7,31,0,23
24,31,0,23
51,31,1,28
79,31,2,13
130,31,4,32
173,31,2,13
192,31,4,32
241,31,4,32
247,31,4,32


In [137]:
dff.sort_values('AGE', ascending=False)

,AGE,PREGNANCIES,COUNTW
289,72,2,1
427,70,4,1
217,67,4,2
342,67,0,1
125,66,2,2
...,...,...,...
124,21,0,111
139,21,1,135
23,21,1,135
27,21,2,103


In [134]:
query = """select age, pregnancies, count(*) over (partition by pregnancies order by age desc) as countw
from diabetes where pregnancies <5 and age>25"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,PREGNANCIES,COUNTW
0,31,1,28
1,33,0,17
2,26,3,43
3,53,2,6
4,30,4,36
...,...,...,...
228,37,1,18
229,52,0,6
230,26,1,56
231,27,2,33


In [138]:
query = """ SELECT Pregnancies,avg(Age) OVER (PARTITION BY pregnancies order by age DESC ) as Countw FROM  diabetes """
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,PREGNANCIES,COUNTW
0,6,57.363636
1,1,40.714286
2,8,46.416667
3,1,27.370370
4,0,46.705882
...,...,...
763,10,63.000000
764,2,35.909091
765,5,43.380952
766,1,54.000000


In [140]:
query = """ select distinct(pregnancies), avg(age) over (partition by pregnancies order by pregnancies desc)as avgr from diabetes"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,PREGNANCIES,AVGR
0,6,39.340000
1,1,27.370370
2,8,45.368421
3,0,27.603604
4,5,39.035088
5,3,29.026667
6,10,42.666667
7,11,44.545455
8,13,44.500000
9,15,43.000000


In [141]:
query = """select distinct(pregnancies), avg(age) over (partition by pregnancies, age order by pregnancies desc) as avgr from diabetes """
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,PREGNANCIES,AVGR
0,6,50.000000
1,1,31.000000
2,8,32.000000
3,0,33.000000
4,5,30.000000
...,...,...
287,11,42.000000
288,6,54.000000
289,3,52.000000
290,6,43.000000


# Rank

In [142]:
query = """ select age, pregnancies, bloodpressure, rank() over (partition by pregnancies order by bloodpressure desc) as rank from diabetes"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,PREGNANCIES,BLOODPRESSURE,RANK
0,47,10,106,1
1,34,10,98,2
2,38,10,86,3
3,56,10,85,4
4,54,10,84,5
...,...,...,...,...
763,26,1,30,132
764,21,1,24,134
765,41,1,0,135
766,46,14,78,1


In [146]:
query = """select age, bloodpressure, pregnancies, DENSE_RANK() OVER (PARTITION BY PREGNANCIES ORDER BY BLOODPRESSURE DESC) AS DENSERANK  from diabetes"""
cursor.execute(query)
out = cursor.fetchall()
dff= pd.DataFrame(out, columns= [i[0] for i in cursor.description])
dff

,AGE,BLOODPRESSURE,PREGNANCIES,DENSERANK
0,47,106,10,1
1,34,98,10,2
2,38,86,10,3
3,56,85,10,4
4,54,84,10,5
...,...,...,...,...
763,26,30,1,31
764,21,24,1,32
765,41,0,1,33
766,46,78,14,1


In [150]:
dff[dff['AGE']==21]

,AGE,BLOODPRESSURE,PREGNANCIES,DENSERANK
206,21,78,3,9
244,21,62,3,17
246,21,60,3,18
352,21,96,0,4
357,21,90,0,6
...,...,...,...,...
741,21,58,1,22
745,21,56,1,23
747,21,55,1,24
753,21,50,1,26


# Group by

In [152]:
query = """select age, pregnancies, outcome, count(*) from diabetes group by age, pregnancies, outcome """
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,PREGNANCIES,OUTCOME,COUNT(*)
0,50,6,1,3
1,31,1,0,1
2,32,8,1,1
3,21,1,0,22
4,33,0,1,2
...,...,...,...,...
380,24,2,1,1
381,33,9,0,1
382,47,1,1,1
383,31,6,1,1


In [153]:
query = """ select age, pregnancies, outcome, count(*) from diabetes
where pregnancies = 3 and outcome =0 
group by age, pregnancies, outcome;"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,PREGNANCIES,OUTCOME,COUNT(*)
0,27,3,0,3
1,22,3,0,8
2,26,3,0,4
3,25,3,0,5
4,30,3,0,3
5,55,3,0,1
6,29,3,0,1
7,21,3,0,3
8,23,3,0,2
9,34,3,0,1


In [155]:
query = """select pregnancies, outcome, count(*) from diabetes 
group by outcome, pregnancies
order by pregnancies desc; """
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,PREGNANCIES,OUTCOME,COUNT(*)
0,17,1,1
1,15,1,1
2,14,1,2
3,13,1,5
4,13,0,5
5,12,0,5
6,12,1,4
7,11,1,7
8,11,0,4
9,10,1,10


In [156]:
query = """select pregnancies, outcome, count(*) from diabetes 
where pregnancies =3 and outcome = 0
group by outcome, pregnancies
order by pregnancies"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,PREGNANCIES,OUTCOME,COUNT(*)
0,3,0,48


In [157]:
query = """ select pregnancies, count(*) as count, avg(age) as avg_age from diabetes group by pregnancies"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,PREGNANCIES,COUNT,AVG_AGE
0,6,50,39.340000
1,1,135,27.370370
2,8,38,45.368421
3,0,111,27.603604
4,5,57,39.035088
5,3,75,29.026667
6,2,103,27.194175
7,4,68,32.779412
8,9,28,44.178571
9,11,11,44.545455


# constraints

In [158]:
cursor.execute("CREATE TABLE aadhardetails (name VARCHAR(30),ADDRESS VARCHAR(20),AADHARID INTEGER PRIMARY KEY)")

In [159]:
cursor.execute("INSERT INTO aadhardetails (name,address,aadharid) VALUES('nethaji','coimbatore',12345678)")

In [160]:
cursor.execute("CREATE TABLE bankacc(name VARCHAR(20),type VARCHAR(2),accno INT PRIMARY KEY,aadhar INT ,FOREIGN KEY (aadhar)REFERENCES aadhardetails(aadharid))")


In [161]:
cursor.execute("INSERT INTO bankacc (name,type,accno,aadhar) VALUES('neteeaji','fd',21234451,12345678)")

In [162]:
cursor.execute("CREATE TABLE pancard(name VARCHAR(20),dob INT,panid VARCHAR(16),aadhar INT ,FOREIGN KEY(aadhar) REFERENCES aadhardetails(aadharid),PRIMARY KEY(panid,aadhar))")


# joins

In [163]:
cursor.execute("CREATE TABLE students(ID INT PRIMARY KEY,NAME VARCHAR(20),PH INT) ")

In [164]:
cursor.execute("INSERT INTO students (ID,NAME,PH) VALUES(1,'prakash',12345)")

cursor.execute("INSERT INTO students (ID,NAME,PH) VALUES(2,'prakash k',123456)")

In [165]:
cursor.execute("CREATE TABLE marks(ID INT ,maths int,phy INT,chem INT) ")

In [167]:
cursor.execute("INSERT INTO marks (ID,maths,phy,chem) VALUES(2,91,81,50)")

In [168]:
 # inner join

query = """ select std.ID, std.name, mark.maths, mark.phy from students as std
INNER JOIN MARKS as mark on std.ID = mark.ID"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,ID,NAME,MATHS,PHY
0,2,prakash k,91,81


In [170]:

query = """ select students.id, students.name, marks.maths, marks.phy, marks.chem 
            from students, marks
            where students.ID= marks.ID;"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,ID,NAME,MATHS,PHY,CHEM
0,2,prakash k,91,81,50


In [173]:

query = """ select students.id, students.name, marks.maths, marks.phy, marks.chem 
            from students, marks
            where students.ID != marks.ID;"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,ID,NAME,MATHS,PHY,CHEM
0,1,prakash,91,81,50


In [174]:
cursor.execute("CREATE TABLE sports(ID INT ,indoor int,outdoor INT,croosfit INT) ")

In [175]:
cursor.execute("INSERT INTO sports (ID,indoor,outdoor,croosfit) VALUES(2,91,80,50)")

In [190]:

query = """ select students.id, students.name, marks.maths, marks.phy, marks.chem, sports.indoor, sports.outdoor, sports.croosfit
from students
INNER JOIN marks on students.id = marks.id 
INNER JOIN sports on students.id = sports.id;"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,ID,NAME,MATHS,PHY,CHEM,INDOOR,OUTDOOR,CROOSFIT
0,2,prakash k,91,81,50,91,80,50


In [192]:
 # outer join

query = """ select std.ID, std.name, mark.maths, mark.phy from students as std
left JOIN MARKS as mark on std.ID = mark.ID"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,ID,NAME,MATHS,PHY
0,1,prakash,NaN,NaN
1,2,prakash k,91.0,81.0


In [193]:

query = """ select std.ID, std.name, mark.maths, mark.phy from students as std
right JOIN MARKS as mark on std.ID = mark.ID"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,ID,NAME,MATHS,PHY
0,2,prakash k,91,81


In [194]:

query = """ select std.ID, std.name, mark.maths, mark.phy from students as std
full JOIN MARKS as mark on std.ID = mark.ID"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,ID,NAME,MATHS,PHY
0,1,prakash,NaN,NaN
1,2,prakash k,91.0,81.0


In [196]:

query = """ select std.ID, std.name, mark.maths, mark.phy from students as std
full outer JOIN MARKS as mark on std.ID = mark.ID"""
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,ID,NAME,MATHS,PHY
0,1,prakash,NaN,NaN
1,2,prakash k,91.0,81.0


In [201]:
query = 'select pregnancies, age, outcome, count(*) from diabetes where pregnancies > 10 group by pregnancies, age, outcome having count(*)<5'
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,PREGNANCIES,AGE,OUTCOME,COUNT(*)
0,11,51,1,2
1,13,57,0,1
2,11,35,0,2
3,13,42,1,1
4,13,45,0,1
5,15,43,1,1
6,17,47,1,1
7,11,40,1,1
8,12,38,1,1
9,12,44,1,1


In [204]:
query = ' select age, sum(outcome) from diabetes group by age having sum(outcome)<20 limit 10'
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,SUM(OUTCOME)
0,50,5
1,31,13
2,32,9
3,21,5
4,30,6
5,29,13
6,53,4
7,54,4
8,57,1
9,59,2


In [207]:
query = ' select top 5 age, sum(outcome) from diabetes group by age having sum(outcome)<20 order by sum(outcome) desc'
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

,AGE,SUM(OUTCOME)
0,25,14
1,31,13
2,29,13
3,41,13
4,22,11


In [ ]:
query = ' select  age, sum(outcome) from diabetes group by age having sum(outcome)<20 order by sum(outcome) desc'
cursor.execute(query)
out = cursor.fetchall()
pd.DataFrame(out, columns= [i[0] for i in cursor.description])

In [1]:
cursor.close()
connections.close()

NameError: name 'cursor' is not defined